In [ ]:
# Standart Imports

import numpy as np
import pandas as pd
import urllib3
import matplotlib.pyplot as plt

from datetime import datetime as dt
from dateutil import parser
from bs4 import BeautifulSoup as soup
from urllib3 import request
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

%matplotlib inline

In [ ]:
def Crawler(forum_name,page_number,num_of_pages):
    
    http = urllib3.PoolManager() 
    page_num = str(page_number) # get current page number of forum
    url = 'https://www.beyondblue.org.au/get-support/online-forums/' + forum_name + '/page/' + page_num 
    
    response = http.request('GET',url) # get desired page
    my_soup = soup(response.data,'lxml') # Create BeautifulSoup object
    
    Post_Count = my_soup.findAll("div",attrs={"class":"postsCount"}) # Find all divs which contain the number of comments for each post
    anchors=my_soup.findAll("a",attrs={"class":"sfforumThreadTitle"}) # Find all the posts's titles
    
    # Create a dictionary to contain the current posts titles and number of comments
    BeyondBlue = {"Post_Title":[],
                 "Num_of_Comments":[],
                 "Url":[]}
    
    # Appending the dict
    for a in anchors:
        BeyondBlue["Post_Title"].append(a.text.encode("ascii","ignore"))
        BeyondBlue["Url"].append(a['href'])
    for number in Post_Count:
        BeyondBlue["Num_of_Comments"].append(int(number.text.strip().encode("ascii","ignore")))
    
    # Create a pandas DataFrame to contain the information
    BeyondBlue = pd.DataFrame(data=BeyondBlue)
    
    # Simple text cleaning to replace all dots and slashes to provide easy acccess to the urls
    BeyondBlue['Url'] = BeyondBlue["Url"].apply(lambda x: x.replace(".",""))
    BeyondBlue['Url'] = BeyondBlue["Url"].apply(lambda x: x.replace("/",""))
    
    # list to contain all the urls
    url_list = np.asarray(BeyondBlue['Url'])
    
    # Iterating through each post's url, we scrape the User's IGN, the date he posted and post's body 
    User_ID = []
    Date = []
    User_Post = []
    for i in range(len(url_list)):
        curr_url = url_list[i] # Get current post's url
        http = urllib3.PoolManager() 
        url = 'https://www.beyondblue.org.au/get-support/online-forums/depression/' + curr_url
        response = http.request('GET',url)
        my_soup = soup(response.data,'lxml') # create a soup object of the current post
        
        UserID = my_soup.findAll("span",attrs={"class":"sfforumUser"}) # Find all the user id's in the current post
        Date_Count = my_soup.findAll("div",attrs={"class":"sfforumPostAge"}) # find the date in the current post
        User_Post.append(my_soup.findAll("div",attrs={"class":"sfContentBlock"})[2].text.encode("ascii","ignore")) # find all the body's text
        
        User_ID.append(UserID[0].text.encode("ascii","ignore")) 
        Date.append(Date_Count[0].text.strip().encode("ascii","ignore"))
        
        print("Finished with {} out of {}".format(page_number*50+i+1,len(url_list)*num_of_pages)) # tracking progress
        
    User_ID = np.asarray(User_ID) 
    Date = np.asarray(Date) 
    User_Post = np.asarray(User_Post) # Convert to numpy arrays to later merge
    
    Post_Data = np.column_stack((User_ID,Date,User_Post)) # Stack the columns along the axes to create the dataframe
    
    Post_Data = pd.DataFrame(data=Post_Data,columns=['User_ID','Date','User_Post']) 
    BeyondBlueCopy = BeyondBlue.copy()
    BeyondBlueCopy = pd.concat([BeyondBlueCopy,Post_Data],axis=1,sort=False) # Concatinating the two dataframes to a single dataframe containing all the information
    BeyondBlueCopy = BeyondBlueCopy[['User_ID','Post_Title','User_Post','Num_of_Comments','Date','Url']] # re-order the dataframe's columns
    BeyondBlueCopy['Date'] = BeyondBlueCopy['Date'].apply(lambda x: parser.parse(x)) # convert date to a numeric representation
    
    return BeyondBlueCopy

In [ ]:
def ScrapeBeyondBlue(forum_name,num_of_pages):
    BeyondBlue = pd.DataFrame() # Create dataframe to hold the returned dataframed and concat it with the current one
    for i in range(num_of_pages): # iterating through the number of pages desired
        BeyondBlue = pd.concat([BeyondBlue,Crawler(forum_name=forum_name,page_number=i,num_of_pages=num_of_pages)],
                              axis=0,sort=False) # call crawler function to get each page's data
        print("Finished with page num:{}".format(i+1)) # track progress
    return(BeyondBlue)

In [ ]:
BeyondBlue = BeyondBlue.reset_index().drop('index',axis=1) # re-order indices
BeyondBlue.head()